# Pulmonary fibrosis analysis - command line code

### Overview of analysis
The goal of this analysis is to compare the gut microbiota of mice in an ABSL2 facility (8th floor), a non-infectious facility (7th floor), and germ-free mice. In each facility, there are mice that have bleomycin-induced pulmonary fibrosis and control mice (saline treatment). The highest degree of pulmonary fibrosis is found in the ABSL2 facility, followed by the non-infectious facility, followed by the germ-free mouse facility. We are testing if the gut microbiota differs between facilities and treatments, and if there are specific taxa of microbes associated with pulmonary fibrosis.

All KneadData, MetaPhlan, and HUMAnN analyses were performed on Vanderbilt University's ACCRE cluster (https://www.vanderbilt.edu/accre/).

### Sequence quality-filtering and trimming
We used KneadData (https://huttenhower.sph.harvard.edu/kneaddata/, v.0.7.6) to quality filter reads, trim low-quality base calls, and remove contaminating sequences. We usd default values for KneadData and the C57BL mouse contaminant database (http://huttenhower.sph.harvard.edu/kneadData_databases/mouse_C57BL_6NJ_Bowtie2_v0.1.tar.gz). As sequencing was paired-end, each read file was run through KneadData individually, and then the resulting cleaned reads were concatenated together for each sample.

kneaddata --input /data/bordenstein_lab/chioma_analysis/sample1_R1_001.fastq.gz --threads 8 \
--reference-db /data/bordenstein_lab/humann_db/mouse_C57BL --output /data/bordenstein_lab/chioma_analysis/kneaddata
kneaddata --input /data/bordenstein_lab/chioma_analysis/sample1_R2_001.fastq.gz --threads 8 \
--reference-db /data/bordenstein_lab/humann_db/mouse_C57BL --output /data/bordenstein_lab/chioma_analysis/kneaddata

cat /data/bordenstein_lab/chioma_analysis/kneaddata/sample1_R1_001_kneaddata.fastq \
/data/bordenstein_lab/chioma_analysis/kneaddata/sample1_R2_001_kneaddata.fastq > \
/Users/bordensteinlab/Bordenstein_Lab/mallott/chioma_analysis/sample1_kneaddata_cat.fastq

### Compositional profiling
We used MetaPhlAn3 (https://huttenhower.sph.harvard.edu/metaphlan3, v.3.0.4) to profile the taxonomic composition of the metagenomes. Default values were used. Output taxonomic profiles were merged for downstream analysis, and species-level relative abundance tables were extracted from the merge profiles.

In [ ]:
metaphlan /data/bordenstein_lab/chioma_analysis/kneaddata/sample1_kneaddata_cat.fastq \
--input_type fastq --nproc 8 --bowtie2db /data/bordenstein_lab/humann_db \
> /data/bordenstein_lab/chioma_analysis/metaphlan_output/sample1_kneaddata_cat.fastq_profile.txt

mv /data/bordenstein_lab/chioma_analysis/kenaddata/*profile.txt /data/bordenstein_lab/chioma_analysis/metaphlan_output

merge_metaphlan_tables.py /data/bordenstein_lab/chioma_analysis/metaphlan_output/*  \
> /data/bordenstein_lab/chioma_analysis/metaphlan_output/merged_abundance_table.txt

grep -E "s__|clade" merged_abundance_table.txt | sed 's/^.*s__//g' | cut -f1,3-70 | sed -e 's/clade_name/body_site/g' \
> merged_abundance_table_species.txt


The vegan package in R was used to calculated beta diversity metrics from the species-level taxonomic relative abundance tables and run PERMANOVAs to test differences in overall community composition (see taxonomic_analysis.Rmd).

LEfSe (https://huttenhower.sph.harvard.edu/galaxy/) was used to identify differentially abundant taxa.

In [ ]:
docker run -it -v /Users:/Users biobakery/lefse bash

format_input.py merged_abundance_table_species_lefse_treatment.txt merged_abundance_species_treatment.lefse \
-u 1 -c 2 -o 1000000

run_lefse.py merged_abundance_species_treatment.lefse merged_abundance_species_treatment.res

plot_res.py --dpi 300 merged_abundance_species_treatment.res merged_abundance_species_treatment.png \
--subclades -1 --max_feature_len 150 --right_space 0.25 --left_space 0.25 --width 12 

format_input.py merged_abundance_table_species_lefse_floor.txt merged_abundance_species_floor.lefse \
-u 1 -c 2 -o 1000000

run_lefse.py merged_abundance_species_floor.lefse merged_abundance_species_floor.res

plot_res.py --dpi 300 merged_abundance_species_floor.res merged_abundance_species_floor.png \
--subclades -1 --max_feature_len 150 --right_space 0.25 --left_space 0.25 --width 12 


### Functional profiling
We used HUMAnN3 (https://huttenhower.sph.harvard.edu/humann3/, v3.0.0) to profile the functional composition of the metagenomes. Default values were used and the translated search was performed using the UniRef90 database. Output taxonomic profiles, gene family tables, and pathway abundance tables for all samples were merged for downstream analysis. Pathway abundance tables were renormalized to relative abundances and gene family tables were regrouped into KEGG Orthogroups and renormalized to copies per million. All pathway abundance and gene family tables were split into stratified and unstratified tables.

In [ ]:
humann --input /data/bordenstein_lab/chioma_analysis/kneaddata/sample1_kneaddata_cat.fastq \
--output /data/bordenstein_lab/chioma_analysis/humann_output/ --threads 8 \
--nucleotide-database /data/bordenstein_lab/humann_db/chocophlan --protein-database \
/data/bordenstein_lab/humann_db/uniref --metaphlan-options "--bowtie2db /data/bordenstein_lab/humann_db"

humann_join_tables --input /data/bordenstein_lab/chioma_analysis/humann_output \
--output /data/bordenstein_lab/chioma_analysis/humann_output/chioma_genefamilies.tsv --file_name genefamilies

humann_join_tables --input /data/bordenstein_lab/chioma_analysis/humann_output \
--output /data/bordenstein_lab/chioma_analysis/humann_output/chioma_pathabundance.tsv --file_name pathabundance

humann_split_stratified_table --input /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_genefamilies.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_genefamilies

humann_split_stratified_table --input /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_pathabundance.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_pathabundance

humann_renorm_table --input /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies_cpm.tsv --units cpm

humann_renorm_table --input /data/bordenstein_lab/chioma_analysis/humann_output/mouse_pathabundance.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/mouse_pathabundance_relab.tsv --units relab

humann_split_stratified_table --input /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_genefamilies_cpm.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_genefamilies_cpm

humann_split_stratified_table --input /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_pathabundance_relab.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/caatinga_pathabundance_relab

humann_regroup_table --input /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies.tsv \
--custom /data/bordenstein/humann_db/utility_mapping/map_ko_uniref90.txt.gz  --output /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies_ko.tsv

humann_renorm_table --input /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies_ko.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies_ko_cpm.tsv --units cpm

humann_split_stratified_table --input /data/bordenstein_lab/chioma_analysis/humann_output/mouse_genefamilies_ko_cpm.tsv \
--output /data/bordenstein_lab/chioma_analysis/humann_output/